<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Computing conservatives and $T^{\mu\nu}$ from primitives

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Duez, Liu, Shapiro, and Stephens. Relativistic Magnetohydrodynamics In Dynamical Spacetimes: Numerical Methods And Tests (2005)](https://arxiv.org/abs/astro-ph/0503420)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**The `compute_conservs_and_Tmunu` function**](#compute_conservs_and_Tmunu)
    1. [*Function Arguments*](#fargs)
        1. [Inputs](#inputs)
        1. [Outputs](#outputs)
    1. [*Computing the Conservatives*](#compute_conservs)
    1. [*Computing $T^{\mu\nu}$*](#compute_tmunu)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

For consistency between primitives and conservatives (or anywhere else where the conservatives need to be computed from the primitives), the `compute_conservs_and_Tmunu` function provides this functionality. It also computes the stress-energy tensor $T^{\mu\nu}$ depending on input parameters. The `enforce_primitive_limits_and_compute_u0` [function](enforce_primitive_limits_and_compute_u0.ipynb) naturally precedes this one, as it applies various limits to the primitives and computes the zeroth component of the 4-velocity $u^0$, which is needed by this function. If the function is being run without `enforce_primitive_limits_and_compute_u0`, the `limit_v_and_compute_u0` function (TODO: link to doc) will directly provide the needed $u^0$.

<a id='compute_conservs_and_Tmunu'></a>

# The `compute_conservs_and_Tmunu` function \[Back to [top](#toc)\]
$$\label{compute_conservs_and_Tmunu}$$

<a id='fargs'></a>

## Function Arguments \[Back to [top](#toc)\]
$$\label{fargs}$$

First, let's look at the arguments of `compute_conservs_and_Tmunu`:
```
void compute_conservs_and_Tmunu(
      const ghl_parameters *restrict params,
      const metric_quantities *restrict metric,
      const primitive_quantities *restrict prims,
      conservative_quantities *restrict cons,
      stress_energy *restrict Tmunu)
```

<a id='inputs'></a>

### Inputs
$$\label{inputs}$$


The parameter used from `params` is `update_Tmunu`, which controls whether $T^{\mu\nu}$ is actually computed.

The metric quantities used from `metric` are the lapse `lapse`, shift (`betax`, `betay`, `betaz`), ADM metric (`adm_gxx`, `adm_gxy`, etc.), inverse metric (`adm_gupxx`, `adm_gupxy`, etc.), 4-metric `g4dn`, inverse lapse `lapseinv` $\left(\alpha^{-1}\right)$, inverse lapse squared `lapseinv2` $\left(\alpha^{-2}\right)$, and conformal factor quantity `psi6` ($\psi^6$). Since auxiliary quantities are used, we recommend use either `GRHayL_enforce_detgtij_and_initialize_metric` or `initialize_metric` to automatically compute these quantities from the ADM quantities.

The primitive quantities used from `prims` are the baryonic density `rho`, pressure `press`, epsilon `eps`, zeroth component of the 4-velocity `u0` ($u^0$), 3-velocity (`vx`, `vy`, `vz`), and the magnetic quantities (`Bx`, `By`, `Bz`). In addition, `entropy` and `Y_e` are used if these conservatives are being used.

<a id='outputs'></a>

### Outputs
$$\label{outputs}$$

The struct `cons` contains the conservative outputs: densitized density $\tilde{\rho}$ (`rho`), densitized energy variable `tau`, and the densitized momentum (`S_x`, `S_y`, `S_z`), the densitized entropy `entropy`, and the densitized electron fraction `Y_e`.

The struct `Tmunu` contains the stress-energy tensor (if `update_Tmunu` is true).

<a id='compute_conservs'></a>

## Computing the Conservatives \[Back to [top](#toc)\]
$$\label{compute_conservs}$$

To compute the conservatives, we first compute a handful of intermediate quantities. We compute the enthalpy

$$
h = 1 + \epsilon + \frac{P}{\rho_b}
$$

and the remainder of the 4-velocity using

$$
u^i = u^0 v^i
$$

We use the helper function `lower_vector` to find $u_\mu$. We then compute
$$b^\mu\equiv \frac{B^i_u}{\sqrt{4 \pi}}$$
(Eq. 31 of https://arxiv.org/abs/astro-ph/0503420) and $b^2$ using the function `compute_smallb_and_b2` (TODO: link to doc). We also compute $b_\mu$, again using `lower_vector`. We after precomputing some intermediate quantities, we're ready to compute the conservatives. These are given by the equations

$$
\tilde{\rho} = \alpha\sqrt{\gamma} \rho_b u^0
$$
$$
\tilde{S}_i = h \tilde{\rho} u_i + \alpha\sqrt{\gamma} \left( u^0 b^2 u_i - b^0 b_i \right)
$$

$$
\tilde{\tau} = \alpha^2\sqrt{\gamma}\left( (\rho_b h + b^2)\left(u^0\right)^2 - \frac{\left(P+\frac{b^2}{2}\right)}{\alpha^2} - \left(b^0\right)^2 - \tilde{\rho} \right)
$$

<a id='compute_tmunu'></a>

## Computing $T^{\mu\nu}$ \[Back to [top](#toc)\]
$$\label{compute_tmunu}$$

While GRHayL does provide the `compute_TUPmunu` function (TODO: link to doc) to directly compute $T^{\mu\nu}$, this would recalculate a large number of quantities already computed for the conservatives. The only lines we need from that function are the ones that actually calculate the stress-energy components from the intermediate quantities we already have. The stress-energy tensor is given by

$$
T^{\mu\nu} = (\rho_b h + b^2)u_\mu u_\nu + \left(P+\frac{b^2}{2}\right)g_{\mu\nu} -b_\mu b_\nu
$$